<a href="https://colab.research.google.com/github/biswajitmohanty/genai/blob/main/Stock_Price_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
import datetime
import math

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN

# Configuration

SHARE = 'AAPL'
SERVICE = 'fred'

PREDICTORS = ['Open']
TARGET = 'Open'

TIMESTEP = 90
START_DATE = datetime.datetime(2010, 1, 1)
END_DATE = datetime.datetime(2019, 9, 30)

START_DATE_TO_PREDICT = datetime.datetime(2019, 10, 1)
END_DATE_TO_PREDICT = datetime.datetime(2019, 10, 31)

N_EPOCHS = 100

df_train = pd.read_csv('Google_Stock_Price_Train.csv')
df_test = pd.read_csv('Google_Stock_Price_Test.csv')

# Data preprocessing

df_train['Date'] = pd.to_datetime(df_train['Date'])
df_test['Date'] = pd.to_datetime(df_test['Date'])

df_train.set_index('Date', inplace=True)
df_test.set_index('Date', inplace=True)

# Keeping only relevant columns
df_train = df_train[PREDICTORS]
df_test = df_test[PREDICTORS]

# Convert training set to numpy array
training_set = df_train.values

# Feature Scaling
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creating data structure with timesteps
X_train, y_train = [], []
target_col_index = df_train.columns.get_loc(TARGET)

for i in range(TIMESTEP, len(training_set)):
    X_train.append(training_set_scaled[i-TIMESTEP:i, :])
    y_train.append(training_set_scaled[i, target_col_index])

X_train, y_train = np.array(X_train), np.array(y_train)

# Building the RNN
regressor = Sequential()

# Adding the first SimpleRNN layer with Dropout
regressor.add(SimpleRNN(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
regressor.add(Dropout(0.2))

# Adding a second SimpleRNN layer with Dropout
regressor.add(SimpleRNN(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

# Adding a third SimpleRNN layer with Dropout
regressor.add(SimpleRNN(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

# Adding a fourth SimpleRNN layer
regressor.add(SimpleRNN(units=50))
regressor.add(Dropout(0.2))

# Output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='adam', loss='mean_squared_error')

# Training the RNN
regressor.fit(X_train, y_train, epochs=N_EPOCHS, batch_size=32)

# Extracting real prices
real_stock_price = df_test[TARGET].values

# Preparing input data for making predictions
df_total = df_train.append(df_test)
inputs = df_total[len(df_total) - len(df_test) - TIMESTEP:][PREDICTORS]

inputs = sc.transform(inputs)

X_test = []
for i in range(TIMESTEP, TIMESTEP+len(df_test)):
    X_test.append(inputs[i-TIMESTEP:i, :])

X_test = np.array(X_test)

# Making the predictions
predicted_stock_price = regressor.predict(X_test)

# Transform predictions back to original scale
temp_matrix = np.zeros((len(predicted_stock_price), len(PREDICTORS)))
temp_matrix[:, target_col_index:target_col_index+1] = predicted_stock_price

predicted_stock_price = sc.inverse_transform(temp_matrix)[:, target_col_index]

# Plot results
df_test['Predicted price'] = predicted_stock_price
df_test[TARGET].plot(figsize=(16,4), legend=True)
df_test['Predicted price'].plot(figsize=(16,4), legend=True)
plt.legend(['Real price', 'Predicted price'])
plt.title(f'Simple RNN - {SHARE} Stock Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 16s 141ms/step - loss: 0.4852
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - loss: 0.2946
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.2296
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.1723
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.1270
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - loss: 0.1184
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - loss: 0.0923
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - loss: 0.0808
Epoch 9/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 74ms/step - loss: 0.0682
Epoch 10/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.0561
Epoch 11/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - loss: 0.0468
Epoch 12/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - loss: 0.0453
Epoch 13/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - loss: 0.0391
Epoch 14/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 5s 74ms/step - loss: 0.0403
Epoch 15/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - 